In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import datetime
import gmaps
import requests
import json
from config import gkey
gmaps.configure(api_key=gkey)

In [ ]:
# Read in all data 
homeprice=pd.read_csv('Inputs/Sale_Prices_Zip.csv')
business=pd.read_csv('Inputs/Licensed_Businesses_WA.csv')
pot_sales=pd.read_csv('Inputs/Sales_and_Tax_Report_WA.csv')
us_states_hp=pd.read_csv('Inputs/Sale_Prices_State.csv')

# Plot Percent Change in Median Home Sales Price Year to Year: States vs. Total US

The following codes cleans the csv that contains median home sales price for every month starting March, 2008 for all states and the total United States, aggregates by year, and plots the percent change. 

In [ ]:
us_states_hp.head()

In [ ]:
us_wa_hp=us_states_hp.loc[(us_states_hp['RegionName']== 'Washington') | (us_states_hp['RegionName']== 'Colorado') | (us_states_hp['RegionName']== 'Oregon')| (us_states_hp['RegionName']== 'United States')]

In [ ]:
us_wa_hp.drop(['RegionID','SizeRank'],axis=1,inplace=True)

In [ ]:
# us_wa_hp

In [ ]:
us_wa_hp.set_index('RegionName',inplace=True)
us_wa_hp=us_wa_hp.transpose()
us_wa_hp.reset_index(inplace=True)
us_wa_hp[['Year','Month']]=us_wa_hp['index'].str.split('-',expand=True)
us_wa_hp

In [ ]:
us_wa_avg=pd.DataFrame(data=[us_wa_hp.groupby('Year')['Washington'].median(),us_wa_hp.groupby('Year')['United States'].median(),us_wa_hp.groupby('Year')['Colorado'].median(),us_wa_hp.groupby('Year')['Oregon'].median()]).transpose()
us_wa_avg

In [ ]:
us_wa_avg['WA_Percent_Change']=us_wa_avg['Washington'].pct_change()*100
us_wa_avg['CO_Percent_Change']=us_wa_avg['Colorado'].pct_change()*100
us_wa_avg['OR_Percent_Change']=us_wa_avg['Oregon'].pct_change()*100
us_wa_avg['US_Percent_Change']=us_wa_avg['United States'].pct_change()*100
us_wa_avg

In [ ]:
testfig,testax=plt.subplots()
x_axis=us_wa_avg.index.tolist()
testax.plot(x_axis,us_wa_avg['Washington'].dropna(),label='Washington',marker='o',color='mediumpurple')
testax.plot(x_axis,us_wa_avg['Oregon'].dropna(),label='Oregon',marker='o',color='darkorange')
testax.plot(x_axis,us_wa_avg['Colorado'].dropna(),label='Colorado',marker='o',color='palevioletred')
testax.plot(x_axis,us_wa_avg['United States'].dropna(),label='Total US',marker='o',color='dimgrey')

testfig.suptitle("Median Home Sales Price in West Coast States in which Cannabis is Legal", fontsize=16, fontweight="bold")
plt.legend(loc='best')
plt.xlabel("Years")
plt.ylabel("Median Home Sales Price ($)")
plt.xticks(x_axis,rotation='vertical')
testax.set_facecolor('whitesmoke')
plt.savefig('Images/west_coast_states.png')

In [ ]:
testfig2,testax2=plt.subplots()
x_axis=[2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
testax2.plot(x_axis,us_wa_avg['WA_Percent_Change'].dropna(),label='Washington',marker='o',color='mediumpurple')
testax2.plot(x_axis,us_wa_avg['OR_Percent_Change'].dropna(),label='Oregon',marker='o',color='darkorange')
testax2.plot(x_axis,us_wa_avg['CO_Percent_Change'].dropna(),label='Colorado',marker='o',color='palevioletred')
testax2.plot(x_axis,us_wa_avg['US_Percent_Change'].dropna(),label='Total US',marker='o',color='dimgrey')

testfig2.suptitle("% Change in Median Home Sales Price", fontsize=16, fontweight="bold")
plt.legend(loc='best')
plt.xlabel("Years")
plt.ylabel("Change from Previous Year (%)")
plt.xticks(x_axis,rotation='vertical')
testax2.set_facecolor('whitesmoke')
plt.show()
plt.savefig('Images/perc_change_home_price.png')

# Build Heat Map of Sales in Washington with Retailer Markers

In [ ]:
business

In [ ]:
business.isna().sum()

In [ ]:
pot_sales

In [ ]:
retailers=business.loc[(business['Type']=='MARIJUANA RETAILER/MEDICAL MARIJUANA ENDORSEMENT') | (business['Type']=='MARIJUANA RETAILER'),: ]


In [ ]:
retailers

In [ ]:
retailers['UBI']=retailers.UBI.astype(str).apply(lambda x: x[:9])
retailers['UBI']=retailers['UBI'].astype(int)

In [ ]:
pot_sales=pot_sales.loc[pot_sales['Total Sales'] != 0,:]
pot_sales.dropna(subset=['UBI'],inplace=True)
# pot_sales.head()

In [ ]:
ret_sales=retailers.merge(pot_sales,on='UBI',how='inner')

In [ ]:
dates=pd.to_datetime(ret_sales['Period Start'],format='%m/%d/%Y')
ret_sales['Sales Month']=dates.apply(lambda x: x.strftime('%Y-%m'))

In [ ]:
ret_sales['Zip']=ret_sales['Zip'].astype(str).apply(lambda x: x[:5]).astype(int)

In [ ]:
sales_by_city=pd.DataFrame(ret_sales.groupby(['City','State'])['Total Sales'].sum())
sales_by_city.reset_index(inplace=True)

In [ ]:
sales_by_city

In [ ]:
url = "https://maps.googleapis.com/maps/api/geocode/json?address="
lat=[]
lng=[]
for i in range(len(sales_by_city)):
    query_url = url + sales_by_city.iloc[i]['City'] + ",+WA&key=" + gkey
    response = requests.get(query_url)
    json = response.json()
    lat.append(json['results'][0]['geometry']['location']['lat'])
    lng.append(json['results'][0]['geometry']['location']['lng'])

sales_by_city['lat']=lat
sales_by_city['lng']=lng

In [ ]:
sales_by_address=pd.DataFrame(ret_sales.groupby(['Address','City','State'])['Total Sales'].sum())
sales_by_address.reset_index(inplace=True)

In [ ]:
lat_mark=[]
lng_mark=[]
for i in range(len(sales_by_address)):
    query_url_mark= url + sales_by_address.iloc[i]['Address']+ ",+" + sales_by_address.iloc[i]['City'] + ",+WA&key=" + gkey
    json_mark = requests.get(query_url_mark).json()
    try:
        lat_mark.append(json_mark['results'][0]['geometry']['location']['lat'])
        lng_mark.append(json_mark['results'][0]['geometry']['location']['lng'])
    except: 
        lat_mark.append('none')
        lng_mark.append('none')

sales_by_address['lat']=lat_mark
sales_by_address['lng']=lng_mark

In [ ]:
locations_marker=sales_by_address[sales_by_address['lat']!='none']

In [ ]:
# Store latitude and longitude in locations
locations = sales_by_city[["lat", "lng"]]
weight=sales_by_city['Total Sales'].astype(float)

heat_layer = gmaps.heatmap_layer(
    locations, weights=weight,dissipating=False,point_radius=0.8)

locations_marker=sales_by_address[sales_by_address['lat']!='none']
locations_marker=locations_marker[['lat','lng']]

marker_layer=gmaps.symbol_layer(locations_marker,fill_color='green',stroke_color='black',scale=2)

fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

# Make Timeseries of Average % Change in Median Home Sales Price in Zip Codes with Marijuana Retailers

In [ ]:
wa_all_hp=homeprice.loc[homeprice['StateName']=='Washington']
wa_all_hp.drop(['RegionID','StateName','SizeRank'],axis=1,inplace=True)

In [ ]:

wa_all_hp.rename(columns={'RegionName':'Zip_MedianHomeSale'},inplace=True)
wa_all_hp.set_index('Zip_MedianHomeSale',inplace=True)
wa_all_hp=wa_all_hp.transpose()
wa_all_hp.head()

In [ ]:
mj_sales_zip=pd.crosstab(ret_sales['Zip'],ret_sales['Sales Month'],values=ret_sales['Total Sales'],aggfunc=np.sum).transpose()
# mj_sales_zip=pd.crosstab(ret_sales['Zip'],ret_sales['Sales Month'],values=ret_sales['Total Sales'],aggfunc=np.sum)
mj_sales_zip.head()

In [ ]:
all_data=mj_sales_zip.join(wa_all_hp,how='outer',lsuffix='_mjsales',rsuffix='_homeprice')
all_data

In [ ]:
df=all_data.filter(like='_',axis=1)
df

In [ ]:
zipcodes_with_mj=df.filter(like='homeprice',axis=1)
zipcodes_with_mj

In [ ]:
list1=list(zipcodes_with_mj)
zipcodes_with_mj.reset_index(inplace=True)
zipcodes_with_mj[['Year','Month']]=zipcodes_with_mj['index'].str.split('-',expand=True)

In [ ]:
zipcodes_with_mj.groupby('Year')['{0}'.format('98103_homeprice')].mean()

In [ ]:
yearavg=pd.DataFrame(index=['2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018'])

In [ ]:

for i in list1:
    yearavg=yearavg.join(zipcodes_with_mj.groupby('Year')['{0}'.format(i)].median(),how='outer')

In [ ]:
yearavg

In [ ]:
yearavgchange=pd.DataFrame(index=['2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018'])
for col in yearavg:
    yearavgchange=yearavgchange.join(yearavg['{0}'.format(col)].pct_change()*100,how='outer',rsuffix='_pctchange')

In [ ]:
yearavgchange.mean(axis=1)

In [ ]:
testfig3,testax3=plt.subplots()
x_axis=[2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
# testax2.plot(x_axis,z,marker='o',color='green',label='5 Change in Median Home Sales Price')
testax3.plot(x_axis,yearavgchange.mean(axis=1).dropna(),label='Zip Codes with Marijuana Retailers',marker='o',color='mediumseagreen')
testax3.plot(x_axis,us_wa_avg['WA_Percent_Change'].dropna(),label='Washington',marker='o',color='mediumpurple')
testax3.plot(x_axis,us_wa_avg['US_Percent_Change'].dropna(),label='Total US',marker='o',color='dimgrey')

testfig3.suptitle("% Change in Median Home Sales Price", fontsize=16, fontweight="bold")
plt.legend(loc='best')
plt.xlabel("Years")
plt.ylabel("Change from Previous Year (%)")
plt.xticks(x_axis,rotation='vertical')
testax3.set_facecolor('whitesmoke')
plt.axvline(x=2014,color='forestgreen')
plt.show()
plt.savefig('Images/Cannabis_Sales_Change.png')